## Практическая работа № 2 "Разграничение доступа"

### Автор: Гордеев Александр Сергеевич КЭ-401

Вариант №2

Количество субъектов (пользователей): 4

Количество объектов доступа: 4

Импортируем необходимые классы и функции из модулей, а также устанавливаем модуль PrettyTable для формирования таблицы доступа

In [57]:
from abc import ABC
from random import randint, seed
from typing import Callable, Dict, Tuple, List, Optional

%pip install prettytable
from prettytable import PrettyTable

Note: you may need to restart the kernel to use updated packages.


Распишем класс со следующими правами в виде битовой маски:

- EMPTY - права отсутствуют
- READ - права для чтения
- WRITE - права на запись
- EXECUTE - права для запуска
- TRANSFER - права для передачи своих прав
- FULL - полные права доступа

In [58]:
class Permissions:
    """Класс, представляющий разрешения."""

    # Значения битов разрешений
    EMPTY = 0b0000
    READ = 0b0001
    WRITE = 0b0010
    EXECUTE = 0b0100
    GRANT = 0b1000
    FULL = READ | WRITE | EXECUTE | GRANT

    @staticmethod
    def to_string(permissions: int) -> str:
        """Преобразует биты разрешений в строковое представление."""
        if permissions == Permissions.EMPTY:
            return "empty"
        if permissions == Permissions.FULL:
            return "full"

        rights = []
        if permissions & Permissions.READ:
            rights.append("read")
        if permissions & Permissions.WRITE:
            rights.append("write")
        if permissions & Permissions.EXECUTE:
            rights.append("execute")
        if permissions & Permissions.GRANT:
            rights.append("grant")
        return ", ".join(rights)

    @staticmethod
    def from_string(permissions: str) -> int:
        """Преобразует строковое представление в биты разрешений."""
        if permissions == "empty":
            return Permissions.EMPTY
        if permissions == "full":
            return Permissions.FULL

        rights = 0
        if "read" in permissions:
            rights |= Permissions.READ
        if "write" in permissions:
            rights |= Permissions.WRITE
        if "execute" in permissions:
            rights |= Permissions.EXECUTE
        if "grant" in permissions:
            rights |= Permissions.GRANT
        return rights

Распишем классы системы:

- Object - базовый класс для объектов
- File - класс файлов, наследуемый от Object
- Device - класс устройств, наследуемый от Object
- User - класс пользователей

In [59]:
class Object(ABC):
    _id_counter = 0

    def __init__(self, name: str):
        self.name = name
        self.id = Object._id_counter
        Object._id_counter += 1

    def __eq__(self, other: object) -> bool:
        return isinstance(other, Object) and self.id == other.id

    def __hash__(self) -> int:
        return hash(self.id)

    def __str__(self) -> str:
        return self.name


class File(Object):
    def __init__(self, name: str):
        super().__init__(name)


class Device(Object):
    def __init__(self, name: str):
        super().__init__(name)


class User:
    def __init__(self, name: str):
        self.name = name
        self.is_admin = False

    def __eq__(self, other: object) -> bool:
        return isinstance(other, User) and self.name == other.name

    def __hash__(self) -> int:
        return hash(self.name)

    def __str__(self) -> str:
        return self.name

Реализуем класс для управления доступом к объектам системы

In [60]:
class AccessManager:
    def __init__(self) -> None:
        """
        Инициализирует класс AccessManager.
        """
        self.users: Dict[str, User] = {}
        self.objects: Dict[str, Object] = {}
        self.access_table: Dict[Tuple[User, Object], Permissions] = {}

    def add_user(self, user: User) -> None:
        """
        Добавляет пользователя в менеджер доступа и выдает разрешения на существующие объекты.

        Аргументы:
            user (User): Добавляемый пользователь.
        """
        self.users[user.name] = user
        for obj in self.objects.values():
            self._grant_permissions(user, obj)

    def add_object(self, obj: Object) -> None:
        """
        Добавляет объект в менеджер доступа и выдает разрешения существующим пользователям.

        Аргументы:
            obj (Object): Добавляемый объект.
        """
        self.objects[obj.name] = obj
        for user in self.users.values():
            self._grant_permissions(user, obj)

    def set_user_admin(self, user: User) -> None:
        """
        Устанавливает пользователя администратором и выдает разрешения на все объекты.

        Аргументы:
            user (User): Пользователь, которого нужно назначить администратором.
        """
        if user not in self.users.values():
            return
        user.is_admin = True
        for obj in self.objects.values():
            self._grant_permissions(user, obj)

    def get_user(self, user_name: str) -> Optional[User]:
        """
        Получает пользователя по имени.

        Аргументы:
            user_name (str): Имя пользователя для поиска.

        Возвращает:
            Optional[User]: Объект пользователя, если найден, в противном случае None.
        """
        return self.users.get(user_name, None)

    def get_object(self, obj_name: str) -> Optional[Object]:
        """
        Получает объект по имени.

        Аргументы:
            obj_name (str): Имя объекта для поиска.

        Возвращает:
            Optional[Object]: Объект, если найден, в противном случае None.
        """
        return self.objects.get(obj_name, None)

    def get_permissions(self, user: User, obj: Object) -> Optional[Permissions]:
        """
        Получает разрешения пользователя на объект.

        Аргументы:
            user (User): Пользователь.
            obj (Object): Объект.

        Возвращает:
            Optional[Permissions]: Разрешения, если найдены, в противном случае None.
        """
        return self.access_table.get((user, obj), None)

    def get_permissions_by_string(
        self, user_name: str, obj_name: str
    ) -> Optional[Permissions]:
        """
        Получает разрешения пользователя на объект по строковым именам.

        Аргументы:
            user_name (str): Имя пользователя.
            obj_name (str): Имя объекта.

        Возвращает:
            Optional[Permissions]: Разрешения, если найдены, в противном случае None.
        """
        return self.get_permissions(self.get_user(user_name), self.get_object(obj_name))

    def get_user_permissions(self, user: User) -> List[Tuple[Object, Permissions]]:
        """
        Получает разрешения пользователя на все объекты.

        Аргументы:
            user (User): Пользователь.

        Возвращает:
            List[Tuple[Object, Permissions]]: Список кортежей (объект, разрешения).
        """
        return [
            (obj, self.access_table.get((user, obj), None))
            for obj in self.objects.values()
        ]

    def grant_permissions_to_user(
        self, owner_name: str, user_name: str, obj_name: str, right: Permissions
    ) -> bool:
        """
        Выдает разрешения пользователю на объект владельца.

        Аргументы:
            owner_name (str): Имя владельца объекта.
            user_name (str): Имя пользователя, которому нужно выдать разрешения.
            obj_name (str): Имя объекта.
            right (Permissions): Права доступа, которые нужно выдать.

        Возвращает:
            bool: True, если разрешения успешно выданы, в противном случае False.
        """
        owner = self.get_user(owner_name)
        user = self.get_user(user_name)
        obj = self.get_object(obj_name)
        if any(i is None for i in [owner, user, obj]):
            return False

        owner_perms = self.get_permissions_by_string(owner_name, obj_name)
        if (
            owner_perms & Permissions.GRANT == 0
            or owner_perms & right == 0
            or (right == Permissions.FULL and owner_perms != right)
        ):
            return False

        user_perms = self.get_permissions_by_string(user_name, obj_name)
        self.access_table[(user, obj)] = user_perms | right
        return True

    def _grant_permissions(self, user: User, obj: Object) -> None:
        """
        Выдает разрешения пользователю на объект.

        Аргументы:
            user (User): Пользователь.
            obj (Object): Объект.
        """
        if user.is_admin:
            self.access_table[(user, obj)] = Permissions.FULL
        else:
            self.access_table[(user, obj)] = randint(1, 15)

Для демонстрации работы AccessManager реализуем класс для взаимодействия пользователя с системой

In [61]:
class CommandParser:
    def __init__(self):
        """
        Инициализирует объект CommandParser.
        """
        self.commands: dict[str, (Callable, str, str)] = {}

    def add_command(
        self, name: str, args: str, description: str, func: Callable
    ) -> None:
        """
        Добавляет команду в CommandParser.

        Аргументы:
            name (str): Название команды.
            args (str): Аргументы, необходимые для команды.
            description (str): Описание команды.
            func (Callable): Функция, которая будет выполнена при вызове команды.
        """
        self.commands[name] = (func, args, description)

    def parse(self, input_str: str) -> None:
        """
        Разбирает входную строку и выполняет соответствующую команду.

        Аргументы:
            input_str (str): Входная строка для разбора.
        """
        if not input_str:
            return

        command, *args = input_str.split()
        if command in self.commands:
            self.commands[command][0](args)
        else:
            print(f"Неизвестная команда: {command}")


class SystemDemo:
    def __init__(self):
        """
        Инициализирует класс SystemDemo с атрибутами access_manager, parser и is_exit.
        """
        self.access_manager = AccessManager()
        self.parser = CommandParser()
        self.is_exit = False
        self._init_parser()

    def run(self) -> None:
        """
        Запускает демонстрацию системы, запрашивая ввод пользователя и выполняя команды.
        """
        user: User = None
        while True:
            while not user:
                inp = input("Введите имя пользователя: ")
                if inp in ["quit", "exit"]:
                    return
                user = self._authenticate(inp)
                if not user:
                    print("Аутентификация не удалась! Попробуйте еще раз...")

            self._greetings_message(user)

            while not self.is_exit:
                inp = input(f"{user} > ")
                if not inp or len(inp) == 0:
                    continue
                self.parser.parse(f"{inp} {user.name}")

            user = None
            self.is_exit = False

    def add_user(self, user: User) -> None:
        """
        Добавляет пользователя в систему.
        """
        self.access_manager.add_user(user)

    def add_admin(self, user: User) -> None:
        """
        Добавляет администратора в систему.
        """
        self.add_user(user)
        self.access_manager.set_user_admin(user)

    def add_object(self, obj: Object) -> None:
        """
        Добавляет объект в систему.
        """
        self.access_manager.add_object(obj)

    def _init_parser(self):
        """
        Инициализация парсера с командами и их описаниями.
        """
        self.parser.add_command("help", "", "Показать эту справку", self._help)
        self.parser.add_command("read", "<object>", "Прочитать объект", self._read)
        self.parser.add_command("write", "<object>", "Записать объект", self._write)
        self.parser.add_command(
            "execute", "<object>", "Исполнить объект", self._execute
        )
        self.parser.add_command(
            "grant",
            "<object> <right> <user>",
            "Предоставить права на объект пользователю",
            self._grant,
        )
        self.parser.add_command("table", "", "Показать таблицу доступа", self._table)
        self.parser.add_command("quit", "", "Выйти из системы", self._quit)

    def _authenticate(self, username: str) -> User:
        """
        Аутентификация пользователя и возврат объекта User.
        """
        return self.access_manager.get_user(username)

    def _greetings_message(self, user: User) -> None:
        """
        Отображение приветственного сообщения и прав пользователя.
        """
        print(f"Аутентифицированный пользователь {user}. Добро пожаловать в систему!")
        print("Ваши разрешения:")
        perms = self.access_manager.get_user_permissions(user)
        for obj, rights in perms:
            print(f"    {obj}: {Permissions.to_string(rights)}")
        print("Введите 'help' для списка команд")

    def _get_usage_info(self, name: str) -> str:
        """
        Получение информации о использовании для определенной команды.
        """
        command = self.parser.commands[name]
        args = command[1]
        return f"{name} {args}"

    def _get_access_info(self, user: User, obj: Object, check: Permissions) -> str:
        """
        Получить информацию о доступе для пользователя и объекта.

        Аргументы:
            user (User): Пользователь, для которого извлекается информация о доступе.
            obj (Object): Объект, для которого извлекается информация о доступе.
            check (Permissions): Проверяемые разрешения.

        Возвращает:
            str: Сообщение о доступе.
        """
        rights = self.access_manager.get_permissions_by_string(user, obj)
        if rights is None:
            return f"Пользователь '{user}' или объект '{obj}' не найден в системе"
        perm = Permissions.to_string(check)
        if rights & check == 0:
            return f"Доступ от '{user}' к объекту '{obj}' с правом '{perm}' запрещен"
        return f"Доступ от '{user}' к объекту '{obj}' с правом '{perm}' разрешен"

    def _help(self, _: list[str]) -> None:
        """
        Отображение доступных команд и их описаний.
        """
        print("Команды:")
        for command in self.parser.commands.items():
            print(
                f"    {command[0]}"
                + (f" {command[1][1]}" if len(command[1][1]) > 0 else "")
                + f" - {command[1][2]}"
            )

    def _read(self, args: list[str]) -> None:
        """
        Читает объект на основе переданных аргументов.

        Аргументы:
            args: Список строк, содержащих объект и исполнителя.
        """
        if len(args) < 2:
            print(f"Использование: {self._get_usage_info('read')}")
            return

        object = args[0]
        executor_user = args[-1]

        access_info = self._get_access_info(executor_user, object, Permissions.READ)
        print(access_info)

    def _write(self, args: list[str]) -> None:
        """
        Записывает в объект на основе переданных аргументов.

        Аргументы:
            args: Список строк, содержащих объект и исполнителя.
        """
        if len(args) < 2:
            print(f"Использование: {self._get_usage_info('write')}")
            return

        object = args[0]
        executor_user = args[-1]

        access_info = self._get_access_info(executor_user, object, Permissions.WRITE)
        print(access_info)

    def _execute(self, args: list[str]) -> None:
        """
        Выполняет объект на основе переданных аргументов.

        Аргументы:
            args: Список строк, содержащих объект и исполнителя.
        """
        if len(args) < 2:
            print(f"Использование: {self._get_usage_info('execute')}")
            return

        object = args[0]
        executor_user = args[-1]

        access_info = self._get_access_info(executor_user, object, Permissions.EXECUTE)
        print(access_info)

    def _grant(self, args: list[str]) -> None:
        """
        Предоставляет разрешения на объект для целевого пользователя.

        Аргументы:
            args: Список строк, содержащих объект, тип разрешения, целевого пользователя и исполнителя.
        """
        if len(args) < 4:
            print(f"Использование: {self._get_usage_info('grant')}")
            return

        object = args[0]
        permission_type = args[1]
        target_user = args[2]
        executor_user = args[-1]

        permissions = self.access_manager.get_permissions_by_string(target_user, object)
        access_info = self._get_access_info(executor_user, object, Permissions.GRANT)
        print(access_info)
        if permissions is None or permissions & Permissions.GRANT == 0:
            return

        success = self.access_manager.grant_permissions_to_user(
            executor_user,
            target_user,
            object,
            Permissions.from_string(permission_type),
        )
        if success:
            print(
                f"Разрешение '{permission_type}' на объект '{object}' "
                + f"от '{executor_user}' предоставлено '{target_user}'"
            )
        else:
            print(
                f"Не удалось предоставить разрешение '{permission_type}' на объект '{object}' "
                + f"от '{executor_user}' для '{target_user}'"
            )

    def _table(self, _: list[str]) -> None:
        """
        Отображает таблицу доступа.
        """
        access_table = self.access_manager.access_table
        users = self.access_manager.users
        objects = self.access_manager.objects

        pretty_table = PrettyTable()
        pretty_table.field_names = ["Пользователь / Объект"] + list(objects.keys())
        for _, user in users.items():
            row = [user.name]
            for _, obj in objects.items():
                permissions = Permissions.to_string(access_table[user, obj])
                row += [permissions]
            pretty_table.add_row(row)
        print(pretty_table)

    def _quit(self, _: list[str]) -> None:
        """
        Завершает программу.
        """
        print("Пока!")
        self.is_exit = True

Запускаем модуль для демонстрации

In [62]:
# Для детерминированного рандома
seed(47)

demo = SystemDemo()

# Добавляем 4 пользователей
demo.add_user(User("alice"))
demo.add_user(User("bob"))
demo.add_user(User("vanya"))
demo.add_admin(User("boss"))

# Добавляем 4 объекта
demo.add_object(File("work"))
demo.add_object(File("report"))
demo.add_object(Device("wifi"))
demo.add_object(Device("camera"))

demo.run()

Аутентифицированный пользователь alice. Добро пожаловать в систему!
Ваши разрешения:
    work: write, execute
    report: read, grant
    wifi: write, execute
    camera: read, write, execute
Введите 'help' для списка команд
+-----------------------+----------------------+--------------+----------------+----------------------+
| Пользователь / Объект |         work         |    report    |      wifi      |        camera        |
+-----------------------+----------------------+--------------+----------------+----------------------+
|         alice         |    write, execute    | read, grant  | write, execute | read, write, execute |
|          bob          |        write         |    grant     | read, execute  | read, write, execute |
|         vanya         | read, write, execute | write, grant |  read, grant   |     write, grant     |
|          boss         |         full         |     full     |      full      |         full         |
+-----------------------+----------------------